# Ceneo Scraper

## Lista kroków "manualnych"

1. Wejdź na stronę produktu / z opiniami o produkcie
2. Dla każdej opini na stronie 
    1. skopiuj opinię
    2. wklej opinię do edytora tekstu
3. Przejdź do kolejnej strony z opiniami
4. Powtarzaj kroki 2-3 dopóki są strony z opiniami 

## Lista krokow Scrapera

1. Wysłanie żądania dostępu do zasobu - strona z opiniami o produkcie 
2. Wydobycie z kodu HTML fragmentów odpowiadających opiniom
3. Dla każdej opinii na stronie
    1. Wydobycie z kodu opini jej składowych do słownika
    2. dodanie słownika reprezętującego pojedyńczą opinię do listy
4. 




## Struktura opini w serwisie Ceneo.pl

| składowa | nazwa | selektor |
|----------|-------|----------|
| identyfikator opinii|opinion_id|["data-entry-id"]|
| autor|author|span.user-post__author-name|
| rekomendacja|recommendation|span.user-post__author-recomendation > em|
| liczba gwiazdek|stars|span.user-post__score-count|
| treść opinii|content|div.user-post__text|
| lista zalet|pros|div.review-feature__title--positives ~ div.review-feature__item|
| lista wad|cons|div.review-feature__title--negatives ~ div.review-feature__item |
| data wystawienia opinii|post_date|span.user-post__published > time:nth-child(1)["datetime"]|
| data zakupu produktu|purchase_date|span.user-post__published > time:nth-child(2)["datetime"]|
| ile osób uznało opinię za przydatną|useful|.vote-yes > span|
| ile osób uznało opinię za nieprzydatną|useless|.vote-no > span|


63717975
124893467
1084811121
114228736
28020568

1. Import bibliotek

In [8]:
import os
import json
import requests
from bs4 import BeautifulSoup

2. Wysłanie żądania dostępu do zasobu (request)

In [2]:
product_code= "28020568"
url= f"https://www.ceneo.pl/{product_code}#tab=reviews"
response = requests.get(url)

3. Wydobycie z kodu HTML fragmentów odpowiadających opiniom

In [3]:
page = BeautifulSoup(response.text, "html.parser")
opinions = page.select("div.js_product-review")

4. Wydobycie z kodu pini jej składowych dla każdej opinii na stronie

In [9]:
all_opinions = []
for opinion in opinions:
    single_opinion = {
        'opinion_id': opinion["data-entry-id"].strip(), 
        'author': opinion.select_one("span.user-post__author-name").text.strip(),
        'recommendation': opinion.select_one("span.user-post__author-recomendation > em").text.strip(),
        'stars': opinion.select_one("span.user-post__score-count").text.strip(),
        'content': opinion.select_one("div.user-post__text").text.strip(),
        'pros': [p.text for p in opinion.select("div.review-feature__title--positives ~ div.review-feature__item")],
        'cons': [c.text for c in opinion.select("div.review-feature__title--negatives ~ div.review-feature__item")],
        'post_date': opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"].strip(),
        'purchase_date': opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"].strip(),
        'useful': opinion.select_one(".vote-yes > span").text.strip(),
        'useless': opinion.select_one(".vote-no > span").text.strip(),
    }
    all_opinions.append(single_opinion)


5. Zapis opinii do pliku JSON

In [10]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")
jf = open(f"opinions/{product_code}.json", "w", encoding="UTF-8")
json.dump(all_opinions, jf, indent=4, ensure_ascii=False)
jf.close()
